In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm
import time
import os
import re
from unidecode import unidecode

In [2]:
# get a list of the most famous rap arists
def get_artist_name_list():
    
    url = "https://www.imdb.com/list/ls054622896/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    artist_headers = soup.find_all('h3', class_='lister-item-header')

    artist_names = [header.find('a').text.strip() for header in artist_headers]

    return artist_names


In [3]:
artist_names = get_artist_name_list()
artist_names[4] = "Notorious B.I.G."
artist_names[0] = "2Pac"

In [4]:
def clean_rapper_name(rapper): # lowercase and remove punctuation    
    
    rapper = rapper.lower()
    rapper = re.sub(r'[^\w\s]', '', rapper)
    
    
    rapper = unidecode(rapper)
    
    rapper_names = rapper.split(" ")
    rapper_full = rapper.replace(" ", "")
    
    rapper_names.insert(0, rapper_full)
    
    return rapper_names
    

def get_az_url(rapper):
    
    
    rapper_names = clean_rapper_name(rapper)
    print(rapper_names)
    
    root = 'https://www.azlyrics.com'
    
    for name in rapper_names:
        
        time.sleep(13)
        
        if rapper[0].isnumeric():
            url = f'{root}/19/{name}.html'
        else:
            url = f'{root}/{name[0].lower()}/{name}.html'
        

        content = requests.get(url)
        
        if content.status_code == 404:
            continue
        else:
            return url
    
    
    return False

get_az_url("Jay-Z")

['jayz', 'jayz']


'https://www.azlyrics.com/j/jayz.html'

In [5]:
def get_rapper_lyrics_list(rapper):
    print(rapper)
    time.sleep(13)
    url = get_az_url(rapper)
    if url is False:
        print("Artist not found.")
        return False 
    
    print(url)
    url_content = requests.get(url)
    
    
    
    if url_content.status_code == 404:
        print("404 status code")
        return False
    
    
    
    html = BeautifulSoup(url_content.text, 'html.parser')
    
    tracks_list = html.find_all("div", {"class": "listalbum-item"})

    lyrics_links = []
    for track in tracks_list:
        if track.find(href=True):
            link = track.find(href=True)['href']
        else:
            continue
        if 'https://www.azlyrics.com' in link:
            lyrics_links.append(link)
        else:
            lyrics_links.append('https://www.azlyrics.com'+link)
    print(f"Found {len(lyrics_links)} songs for {rapper}")
    return lyrics_links


In [6]:
all_links = {}
# tupac, 
for rapper_name in artist_names:
    
    lyric_links = get_rapper_lyrics_list(rapper_name)
    
    if lyric_links:
        all_links[rapper_name] = lyric_links

2Pac
['2pac', '2pac']
https://www.azlyrics.com/19/2pac.html
Found 275 songs for 2Pac
Jay-Z
['jayz', 'jayz']
https://www.azlyrics.com/j/jayz.html
Found 312 songs for Jay-Z
Nas
['nas', 'nas']
https://www.azlyrics.com/n/nas.html
Found 408 songs for Nas
Eminem
['eminem', 'eminem']
https://www.azlyrics.com/e/eminem.html
Found 414 songs for Eminem
Notorious B.I.G.
['notoriousbig', 'notorious', 'big']
https://www.azlyrics.com/n/notorious.html
Found 130 songs for Notorious B.I.G.
Scarface
['scarface', 'scarface']
https://www.azlyrics.com/s/scarface.html
Found 188 songs for Scarface
André 3000
['andre3000', 'andre', '3000']
https://www.azlyrics.com/a/andre3000.html
Found 15 songs for André 3000
DMX
['dmx', 'dmx']
https://www.azlyrics.com/d/dmx.html
Found 259 songs for DMX
Jadakiss
['jadakiss', 'jadakiss']
https://www.azlyrics.com/j/jadakiss.html
Found 164 songs for Jadakiss
Lil' Wayne
['lilwayne', 'lil', 'wayne']
https://www.azlyrics.com/l/lilwayne.html
Found 824 songs for Lil' Wayne
Snoop Dogg

In [ ]:
# idea: run a clustering algorithm on the artists
# run a different llm on each of the clusters

In [1]:
import pickle
filename = "song_links.pkl"
    
    
with open(filename, "rb") as f:
    links = pickle.load(f)


In [4]:
n = 0 
a = 0
for v in links.values():
    n += len(v)
    a += 1
print(a, n)

44 11205


In [41]:
import random
data_list = {'Rapper':[],'Title':[], 'Lyrics':[]}

for rapper, lyrics_links in links.items():
    for url in tqdm(lyrics_links, desc = f"Gathering songs lyrics of {rapper}"):
        
        sleep_time = 13
        time.sleep(sleep_time) # wait between requests to avoid being blocked from the site
        
        
        try:
            lyrics_page = requests.get(url)

            if lyrics_page.status_code != 200:
                print("faulty request: ", lyrics_page.status_code)

            lyrics_html = BeautifulSoup(lyrics_page.text, 'html.parser')

            title = lyrics_html.select('h1')[0].text.strip().split('"')[1]
            lyrics = max(lyrics_html.get_text().split('\n\n\n\n'), key=len).split('\n\n\n\r\n')[-1]


            tags = re.findall("\[.*?\]", lyrics)
            for tag in tags:
                lyrics = lyrics.replace(tag,'')
        except Exception as e:
            print(e)
            continue
        
        data_list['Rapper'].append(rapper)
        data_list['Title'].append(title)  
        data_list['Lyrics'].append(lyrics)
    
    
    
    df = pd.DataFrame(data_list)
    df.to_csv(f"lyric_files/{rapper}.csv")
    
    
    data_list = {'Rapper':[],'Title':[], 'Lyrics':[]}



Gathering songs lyrics of Rakim: 100%|██████████| 53/53 [12:03<00:00, 13.65s/it]
Gathering songs lyrics of Tip 'T.I.' Harris:  56%|▌| 170/304 [38:50<31:04, 13.91

faulty request:  403
list index out of range


Gathering songs lyrics of Tip 'T.I.' Harris:  56%|▌| 171/304 [39:05<31:10, 14.07

faulty request:  403
list index out of range


Gathering songs lyrics of Tip 'T.I.' Harris:  57%|▌| 172/304 [39:19<31:20, 14.25

...

faulty request:  403
list index out of range


Gathering songs lyrics of J. Cole:   0%|      | 1/270 [00:13<1:02:23, 13.92s/it]

faulty request:  403
list index out of range


Gathering songs lyrics of J. Cole:   0%|      | 1/270 [00:19<1:27:17, 19.47s/it]


KeyboardInterrupt: 